https://arxiv.org/pdf/1704.04110.pdf

https://archive.ics.uci.edu/ml/datasets/ElectricityLoadDiagrams20112014

https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/deepar_electricity/DeepAR-Electricity.ipynb

TODO:

- sample by $v_{i}$

In [159]:
import pandas as pd

d = pd.read_csv('LD2011_2014.txt', sep=';', thousands=',', index_col=0, )
d.index = pd.to_datetime(d.index)
d.shape

(140256, 370)

In [160]:
d.head()

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_361,MT_362,MT_363,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369,MT_370
2011-01-01 00:15:00,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2011-01-01 00:30:00,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2011-01-01 00:45:00,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2011-01-01 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2011-01-01 01:15:00,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0


In [161]:
d = d.resample('H', how='sum')
d.head()

/Users/dgates/venvs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  """Entry point for launching an IPython kernel.


,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_361,MT_362,MT_363,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369,MT_370
2011-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2011-01-01 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2011-01-01 02:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2011-01-01 03:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0
2011-01-01 04:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0,0,0,0,0,0


In [162]:
d.shape

(35065, 370)

In [163]:
# hopw many training examples total
import numpy as np
sum(d.apply(lambda s: np.argwhere(s > 0).reshape(-1).max() - np.argwhere(s > 0).reshape(-1).min()))

/Users/dgates/venvs/py3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


10463878

In [164]:
# all observations are 1 hour intervals
pd.Series(d.index).diff().unique()

array([        'NaT', 3600000000000], dtype='timedelta64[ns]')

In [165]:
for c in d.columns:
    d[c] = d[c].astype(float)
d.dtypes

MT_001    float64
MT_002    float64
MT_003    float64
MT_004    float64
MT_005    float64
           ...   
MT_366    float64
MT_367    float64
MT_368    float64
MT_369    float64
MT_370    float64
Length: 370, dtype: object

In [166]:
d.index.min(), d.index.max()

(Timestamp('2011-01-01 00:00:00'), Timestamp('2015-01-01 00:00:00'))

In [167]:
d.shape

(35065, 370)

In [168]:
d.sum(axis=0)

MT_001    1.899547e+19
MT_002    2.755961e+19
MT_003    2.490666e+19
MT_004    5.058810e+19
MT_005    4.527706e+19
              ...     
MT_366    4.462558e+19
MT_367    5.406011e+19
MT_368    3.381737e+19
MT_369    6.853980e+19
MT_370    1.331800e+19
Length: 370, dtype: float64

In [11]:
d.isnull().any().any()

False

# feature engineering

In [282]:
import numpy as np


# 7 days of context to predict one day of outcomes
condition_length = 24 * 7
prediction_length = 24


def featurize(s, id):
    name = s.name
    s = drop_padded_values(s)
    df = convert_series_to_frame(s)
    df = add_categorical_features(df, id)
    df = add_scaling_factor(df)
    df['name'] = name
    return df

def drop_padded_values(s):
    nonzeros = s.to_numpy().nonzero()[0]
    return s.iloc[nonzeros[0]:nonzeros[-1]]

def convert_series_to_frame(s):
    return s.to_frame().rename({s.name: 'z'}, axis=1)

def add_categorical_features(df, id):
    df['id'] = id
    df['day_of_week'] = df.index.dayofweek
    df['hour'] = df.index.hour
    df['month'] = df.index.month
    df['week_of_year'] = df.index.weekofyear
    df['age'] = (df.index - df.index.values[0]).total_seconds() / 3600
    return df

def add_scaling_factor(df):
    z = df.z + 100
    df['scale_factor'] = 1 + z.rolling(condition_length).sum() - 100 * condition_length
    return df
    
%time all_time_series = pd.concat(featurize(d.iloc[:, i], i) for i in range(d.shape[-1]))

CPU times: user 7.98 s, sys: 3.32 s, total: 11.3 s
Wall time: 13 s


In [283]:
all_time_series.head()

,z,id,day_of_week,hour,month,week_of_year,age,scale_factor,name
2012-01-01 00:00:00,8.121827e+14,0,6,0,1,52,0.0,NaN,MT_001
2012-01-01 01:00:00,1.192893e+15,0,6,1,1,52,1.0,NaN,MT_001
2012-01-01 02:00:00,1.446701e+15,0,6,2,1,52,2.0,NaN,MT_001
2012-01-01 03:00:00,8.629442e+14,0,6,3,1,52,3.0,NaN,MT_001
2012-01-01 04:00:00,5.329949e+14,0,6,4,1,52,4.0,NaN,MT_001


In [284]:
all_time_series.scale_factor.fillna(1e-9, inplace=True)

In [285]:
assert all_time_series.scale_factor.lt(0).sum() == 0, all_time_series.scale_factor.lt(0).sum()

AssertionError: 17454

In [269]:
all_time_series['sample_weight'] = all_time_series.scale_factor / all_time_series.scale_factor.sum()

In [270]:
all_time_series.head()

,z,id,day_of_week,hour,month,week_of_year,age,scale_factor,name,sample_weight
2012-01-01 00:00:00,8.121827e+14,0,6,0,1,52,0.0,1.000000e-09,MT_001,4.208852e-34
2012-01-01 01:00:00,1.192893e+15,0,6,1,1,52,1.0,1.000000e-09,MT_001,4.208852e-34
2012-01-01 02:00:00,1.446701e+15,0,6,2,1,52,2.0,1.000000e-09,MT_001,4.208852e-34
2012-01-01 03:00:00,8.629442e+14,0,6,3,1,52,3.0,1.000000e-09,MT_001,4.208852e-34
2012-01-01 04:00:00,5.329949e+14,0,6,4,1,52,4.0,1.000000e-09,MT_001,4.208852e-34


In [281]:
(d.iloc[:, 14]).rolling(condition_length).sum().min()

-862.0

In [271]:
np.random.choice(all_time_series.shape[0], p=all_time_series.sample_weight)

8650169

In [148]:
import numpy as np

# TODO: add age feature and "item" feature

# 7 days of context to predict one day of outcomes
condition_length = 24 * 7
prediction_length = 24

categorical_features = [
    'day_of_week', 'hour', 'month', 'week_of_year', 'age',
]

def get_categorical_features(d):
    return d[categorical_features].values[np.newaxis, :, :]

def get_time_series_features(d):
    return d.z.T.values[:, :, np.newaxis]

def make_id_feature(ids, n):
    return np.tile(ids, (n, 1))

def feature_extraction(d, condition_length=condition_length):
    d_condition = d.iloc[:condition_length]
    d_prediction = d.iloc[condition_length:-1]
    d_y = d.iloc[condition_length + 1:]
    
    x_condition = get_categorical_features(d_condition)
    x_prediction = get_categorical_features(d_prediction)
    
    z_condition = get_time_series_features(d_condition)
    z_prediction = get_time_series_features(d_prediction)
    y = get_time_series_features(d_y)
    
    scale_factor = 1 + z_condition.sum(axis=(1, 2)).reshape((-1, 1, 1))
    
    return {
        'id_condition': make_id_feature(d.id, condition_length),
        'id_prediction': make_id_feature(d.id, len(d_prediction)),
        'x_condition': x_condition.astype(float),
        'x_prediction': x_prediction.astype(float),
        'z_prediction': z_prediction / scale_factor,
        'z_condition': z_condition / scale_factor,
        'scale_factor': scale_factor
    }, y / scale_factor

def get_training_example(d):
    i = np.random.randint(370)  # TODO: don't hard code number of clients
    first_nonzero = np.argwhere(d.iloc[:, i] > 0)[0][0]
    t_0 = np.random.randint(np.argwhere(d.iloc[:, 0] > 0)[0][0], len(d) - (prediction_length + 1))
    d_sample = d[t_0:t_0 + condition_length + prediction_length + 1]  # TODO: parameterize
    return feature_extraction(d_sample[[d_sample.columns[i]] + categorical_features], i)

def get_batch(d, batch_size):
    training_examples = [get_training_example(d) for _ in range(batch_size)]
    keys = training_examples[0][0].keys()
    x = {k: np.concatenate([t[0][k] for t in training_examples]) for k in keys}
    y = np.concatenate([t[1] for t in training_examples])
    return x, y


In [20]:
get_training_example(d)

/Users/dgates/venvs/py3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


({'id_condition': array([[69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69,
          69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69,
          69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69,
          69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69,
          69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69,
          69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69,
          69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69,
          69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69,
          69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69,
          69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69,
          69, 69, 69, 69, 69, 69, 69, 69]]),
  'id_prediction': array([[69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69,
          69, 69, 69, 69, 69, 69, 69, 69]]),
  'scale_factor': array([[[2.33806121e+17]]]),
  'x_condition':

In [21]:
b_x, b_y = get_batch(d, 32)
b_x['x_condition'].shape, b_x['z_condition'].shape, b_x['z_prediction'].shape, b_y.shape

/Users/dgates/venvs/py3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


((32, 168, 5), (32, 168, 1), (32, 24, 1), (32, 24, 1))

In [22]:
b_x['scale_factor'].shape

(32, 1, 1)

In [23]:
b_x.keys()

dict_keys(['id_condition', 'id_prediction', 'x_condition', 'x_prediction', 'z_prediction', 'z_condition', 'scale_factor'])

In [24]:
assert (b_x['z_prediction'][:, 1:, :] == b_y[:, :-1, :]).all()

# model

In [29]:
import math
import tensorflow as tf

pi = tf.constant(math.pi, dtype='float64')

def guassian_likelihood(z, model_outputs):
    mu, sigma = model_outputs[:, :, 0], model_outputs[:, :, 1]
    # need to keep three dimensions when we slice here
    mu, sigma = tf.keras.backend.expand_dims(mu), tf.keras.backend.expand_dims(sigma) + 1e-6
    return (1. / tf.math.sqrt(2 * pi * sigma ** 2)) * tf.exp(-1 * (((z - mu) ** 2) / ((2 * sigma ** 2))))


def guassian_log_likelihood_loss(z, model_outputs):
    return -tf.math.log(guassian_likelihood(z, model_outputs) + 1e-9)


def smart_log_likelihood_loss(z, model_outputs):
    mu, sigma = model_outputs[:, :, 0], model_outputs[:, :, 1]
    # need to keep three dimensions when we slice here
    mu, sigma = tf.keras.backend.expand_dims(mu), tf.keras.backend.expand_dims(sigma) + 1e-6
    return tf.math.log(sigma) + (((z - mu) ** 2) / (2 * sigma ** 2))


def _make_inputs(condition_length, prediction_length, time_cat_dims):
    id_condition = tf.keras.Input((condition_length,), name='id_condition')
    id_prediction = tf.keras.Input((prediction_length,), name='id_prediction')

    x_condition_input = tf.keras.Input((condition_length, time_cat_dims), name='x_condition')
    x_prediction_input = tf.keras.Input((prediction_length, time_cat_dims), name='x_prediction')
    
    z_condition_input = tf.keras.Input((condition_length, 1), name='z_condition')
    z_prediction_input = tf.keras.Input((prediction_length, 1), name='z_prediction')
    
    scale_factor_input = tf.keras.Input((1, 1), name='scale_factor')
    
    return id_condition, id_prediction, x_condition_input, x_prediction_input, z_condition_input, z_prediction_input, scale_factor_input


def _make_encoder_decoder(encoder_inputs, decoder_inputs, lstm_layers, lstm_units):
    # make encoder first
    encoder_outputs = encoder_inputs
    for i in range(lstm_layers):
        encoder = tf.keras.layers.LSTM(lstm_units, return_state=True, return_sequences=True,
                                       unit_forget_bias=1.0,  # this is the default, but explicitly mentioned in the paper
                                       name=f'encoder_{i}')
        encoder_outputs, *encoder_state  = encoder(encoder_outputs)

    # now make decoder, sharing encoder state on the first layer
    decoder_outputs = decoder_inputs
    for i in range(lstm_layers):
        decoder = tf.keras.layers.LSTM(lstm_units, return_sequences=True,
                                       unit_forget_bias=1.0,  # this is the default, but explicitly mentioned in the paper
                                       name=f'decoder_{i}')
        if i == 0:
            decoder_outputs = decoder(decoder_outputs, initial_state=encoder_state)
        else:
            decoder_outputs = decoder(decoder_outputs)
    return decoder_outputs

def _make_distribution_params(sequences, scale, W_mu_activation, W_sigma_activation):
    mu = tf.keras.layers.Dense(1, activation=W_mu_activation)(sequences)
    sigma = tf.keras.layers.Dense(1, activation=W_sigma_activation)(sequences)
    
    mu = tf.keras.layers.Multiply()([mu, scale])
    sigma = tf.keras.layers.Multiply()([sigma, 1 / tf.math.sqrt(scale)])
    return tf.keras.layers.Concatenate()([mu, sigma])
        

def DeepAR(condition_length, prediction_length, time_cat_dims, item_input_embedding_dim=370, item_output_embedding_dim=20,
           lstm_layers=3, lstm_units=40, W_mu_activation='linear', W_sigma_activaton='softplus'):

    id_condition, id_prediction, x_condition_input, x_prediction_input, z_condition_input, z_prediction_input, scale_factor = \
        _make_inputs(condition_length, prediction_length, time_cat_dims)

    condition_embedding = tf.keras.layers.Embedding(output_dim=item_output_embedding_dim, input_dim=item_input_embedding_dim,
                                                    input_length=condition_length)(id_condition)
    prediction_embedding = tf.keras.layers.Embedding(output_dim=item_output_embedding_dim, input_dim=item_input_embedding_dim,
                                                     input_length=prediction_length)(id_prediction)
    condition_inputs = tf.keras.layers.Concatenate()([
        tf.keras.layers.BatchNormalization(trainable=False)(x_condition_input),
        z_condition_input,
        condition_embedding])
    prediction_inputs = tf.keras.layers.Concatenate()([
        tf.keras.layers.BatchNormalization(trainable=False)(x_prediction_input),
        z_prediction_input,
        prediction_embedding])

    decoder_outputs = _make_encoder_decoder(condition_inputs, prediction_inputs, lstm_layers, lstm_units)

    z_distribution_params = _make_distribution_params(decoder_outputs, scale_factor, W_mu_activation, W_sigma_activaton)

    return tf.keras.models.Model(
        inputs=[
            scale_factor,
            id_condition, id_prediction,
            x_condition_input, x_prediction_input,
            z_condition_input, z_prediction_input],
        outputs=[z_distribution_params])

tf.keras.backend.set_floatx('float64')
model = DeepAR(condition_length, prediction_length, len(categorical_features), lstm_layers=3, W_mu_activation='softplus')
adam = tf.keras.optimizers.Adam(learning_rate=.001)
model.compile(loss=guassian_log_likelihood_loss, optimizer=adam)

In [30]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_condition (InputLayer)        [(None, 168, 5)]     0                                            
__________________________________________________________________________________________________
id_condition (InputLayer)       [(None, 168)]        0                                            
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 168, 5)       20          x_condition[0][0]                
__________________________________________________________________________________________________
z_condition (InputLayer)        [(None, 168, 1)]     0                                            
____________________________________________________________________________________________

In [31]:
last = [None]
def gen(d, batch_size):
    while True:
        try:
            x_batch, y_batch = get_batch(d, batch_size)
            last[0] = x_batch, y_batch
            y_batch = y_batch * x_batch['scale_factor']
            yield x_batch, y_batch
        except Exception:
            pass

In [32]:
callbacks = model.fit(
    gen(d, 64),
    steps_per_epoch=3000,
    epochs=50,
    callbacks=[tf.keras.callbacks.TerminateOnNaN()])

/Users/dgates/venvs/py3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


Train for 3000 steps
Epoch 1/50
   1/3000 [..............................] - ETA: 7:40:53 - loss: 20.4154

/Users/dgates/venvs/py3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


  54/3000 [..............................] - ETA: 57:55 - loss: 19.1637

KeyboardInterrupt: 

In [ ]:
x_test, y_test = next(gen(d, 64))   # TODO: not true test cases
p = model.predict(x_test)

In [ ]:
import scipy.stats as ss
import matplotlib.pyplot as plt

i = np.random.randint(len(p))

params_it = p[i]
dist_it = ss.norm(params_it[:, 0], params_it[:, 1])
mean_it = dist_it.mean()
low_it = dist_it.ppf(0.1)
high_it = dist_it.ppf(0.9)
y_it = y_test[i].reshape(-1)
n = y_test.shape[-2]

fig, ax = plt.subplots(figsize=(15, 5))
plt.plot(np.arange(n), y_it, color='b')
# plt.errorbar(np.arange(n), mean_it, color='r')
plt.errorbar(np.arange(n)[1:], mean_it[1:], color='r', yerr=[np.abs(low_it)[1:], high_it[1:]])
# plt.plot(np.arange(n), low_it)
# plt.plot(np.arange(n), high_it)

In [ ]:
params_it[0, 0], params_it[0, 1], dist_it.ppf(0.1)[0]